In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [3]:
# answer goes here
nba = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv"
)
nba.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [4]:
# answer goes here
# remove players with GS < 20
nba = nba[nba["GS"] >= 20]
nba.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,...,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,...,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4


<IPython.core.display.Javascript object>

In [5]:
# remove players with mp >= 10
nba = nba[nba["MP"] >= 10]
nba.head()

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,...,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,...,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,...,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4


<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [6]:
# answer goes here
col = ["PTS", "TRB", "AST", "STL", "BLK", "TOV", "Age"]
nba = nba[col]
nba.head()

,PTS,TRB,AST,STL,BLK,TOV,Age
3,13.9,9.5,1.6,1.5,1.0,1.7,25
4,8.9,7.3,2.2,0.9,0.8,1.5,21
7,21.3,9.2,2.4,0.5,1.3,1.8,33
10,10.9,8.4,1.4,0.5,1.5,1.3,20
12,9.4,7.5,1.3,0.8,0.4,0.9,28


<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [7]:
# answer goes here
scaler = StandardScaler()
scaled = scaler.fit_transform(nba)

<IPython.core.display.Javascript object>

Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [8]:
# answer goes here
k_means = KMeans(n_clusters=3)
k_means.fit(scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

<IPython.core.display.Javascript object>

In [9]:
pd.DataFrame(k_means.cluster_centers_, columns=nba.columns)

,PTS,TRB,AST,STL,BLK,TOV,Age
0,1.077700,-0.002931,1.436138,1.033012,-0.213484,1.342999,-0.035427
1,0.084387,1.240259,-0.468340,-0.360529,1.291835,-0.008272,-0.141092
2,-0.478424,-0.514630,-0.395988,-0.274996,-0.449466,-0.549021,0.073255


<IPython.core.display.Javascript object>

In [10]:
df = pd.DataFrame(
    scaler.inverse_transform(k_means.cluster_centers_), columns=nba.columns
)
df.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,Age
0,19.501657,5.289503,5.895028,1.350276,0.486188,2.788950,26.320442
1,13.724044,8.528962,1.912568,0.771038,1.231694,1.673770,25.896175
2,10.450455,3.956136,2.063864,0.806591,0.369318,1.227500,26.756818


<IPython.core.display.Javascript object>

In [11]:
# nba["cluster_label"] = k_means.labels_
# sns.pairplot(nba, hue="cluster_label")
# plt.show()

<IPython.core.display.Javascript object>

Experiment with different values of K. Do any further interesting archetypes come out?

In [12]:
# answer goes here
k_means = KMeans(n_clusters=5)
k_means.fit(scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

<IPython.core.display.Javascript object>

In [13]:
pd.DataFrame(k_means.cluster_centers_, columns=nba.columns)

,PTS,TRB,AST,STL,BLK,TOV,Age
0,0.422339,1.553213,-0.367671,-0.272346,1.688629,0.224426,-0.087561
1,1.247256,0.154022,1.682697,1.210961,-0.165709,1.556503,0.148914
2,-0.594277,-0.406754,-0.430344,-0.493044,-0.401476,-0.629393,1.120764
3,-0.849072,-0.261758,-0.756095,-0.672016,-0.060308,-0.813735,-0.746872
4,0.192132,-0.408633,0.135911,0.379425,-0.448793,0.107908,-0.478283


<IPython.core.display.Javascript object>

In [14]:
df = pd.DataFrame(
    scaler.inverse_transform(k_means.cluster_centers_), columns=nba.columns
)
df.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,Age
0,15.689744,9.344444,2.123077,0.807692,1.428205,1.865812,26.111111
1,20.487879,5.698485,6.410606,1.424242,0.509848,2.965152,27.060606
2,9.776596,4.237234,1.992021,0.715957,0.393085,1.161170,30.962766
3,8.294578,4.615060,1.310843,0.641566,0.562048,1.009036,23.463855
4,14.350746,4.232338,3.176119,1.078607,0.369652,1.769652,24.542289


<IPython.core.display.Javascript object>